In [12]:
import tensorflow as tf
# print(tf.__version__)


In [ ]:
import kagglehub
import os
import shutil
# Download latest version
path = kagglehub.dataset_download("imsparsh/flowers-dataset")

print("Path to dataset files:", path)
# Copiar recursivamente todas las carpetas y archivos de 'path' a 'dataset' sin función recursiva
if not os.path.exists("dataset"):
    os.makedirs("dataset")
for root, dirs, files in os.walk(path):
    # Construir la ruta de destino correspondiente
    dest_dir = os.path.join("dataset", os.path.relpath(root, path))
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for file in files:
        src_file = os.path.join(root, file)
        dst_file = os.path.join(dest_dir, file)
        shutil.copy2(src_file, dst_file)


Path to dataset files: C:\Users\isard\.cache\kagglehub\datasets\imsparsh\flowers-dataset\versions\2


In [21]:
# Codigo general
# Si quieres ver como lo programamos, revisa el video donde lo explico
# todo desde cero: https://youtu.be/iX_on3VxZzk
import keras
import tensorflow as tf
import numpy as np

# No need for 'import keras' separately when using TensorFlow 2.x+

# print(f"Using TensorFlow version: {tf.__version__}") # Good practice to check

# Input data
celsius = np.array([-40, -10, 0, 8, 15, 22, 38], dtype=float)
fahrenheit = np.array([-40, 14, 32, 46, 59, 72, 100], dtype=float)

# Define the model using tf.keras
# Layer definitions are clearer when defined inline or assigned to descriptive variables
model = tf.keras.Sequential([
    keras.layers.Dense(units=3, input_shape=[1], name="Hidden_Layer_1"), # Added names for clarity
    keras.layers.Dense(units=3, name="Hidden_Layer_2"),
    keras.layers.Dense(units=1, name="Output_Layer")
])

# Compile the model using tf.keras
# Using Adam optimizer directly from tf.keras.optimizers
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), # Note: 0.01 was in the original, 0.1 is often a better starting point for Adam, but adjust as needed
    loss='mean_squared_error'
)

# Print model summary (optional but helpful)
print("\nModel Summary:")
model.summary()

# Train the model
print("\nComenzando entrenamiento...")
# It's good practice to store the history object
historial = model.fit(celsius, fahrenheit, epochs=300, verbose=False) # verbose=False suppresses output during training
print("Modelo entrenado!")

# You can optionally plot the loss history afterwards
# import matplotlib.pyplot as plt
# plt.xlabel("# Epoca")
# plt.ylabel("Magnitud de pérdida")
# plt.plot(historial.history["loss"])
# plt.show()

# Example prediction
print("\nRealizando una predicción:")
prediction_celsius = 100.0
predicted_fahrenheit = model.predict([prediction_celsius])
print(f"{prediction_celsius}°C son aproximadamente {predicted_fahrenheit[0][0]:.2f}°F según el modelo.")
# Actual conversion: (100 * 9/5) + 32 = 212


AttributeError: module 'tensorflow' has no attribute 'ones'

In [ ]:
resultado = modelo.predict([100.0])
print("Prediccion: 100 grados celsius son " + str(resultado) + " fahrenheit!")

In [ ]:
#Exportar el modelo en formato h5
modelo.save('celsius_a_fahrenheit.h5')

In [ ]:
#Crear carpeta donde se colocaran los archivos resultantes


In [ ]:
import subprocess
subprocess.run(["mkdir", "carpeta_salida"])
#Convertir el modelo a tensorflow.js
subprocess.run(["tensorflowjs_converter", "--input_format", "keras", "celsius_a_fahrenheit.h5", "carpeta_salida"])

In [ ]:
#Confirmar que en la carpeta de salida se hayan generado los archivos. Deben aparecer archivos "bin" y "json"
# !ls carpeta_salida

In [ ]:
#Para descargarlos, da clic del lado izquierdo en el icono de la carpeta
#y expande carpeta_salida. En los archivos utiliza los 3 puntos para descargarlos